In [ ]:
# -*- coding: utf-8 -*-
'''
Created on 2015-10-19 
@author: 陈逸逵 
'''
import math
import random
import pickle
from os import path
import xlrd
import time

#calculate the overall average
def Average(trainData):
    result = 0.0
    cnt = 0
    for row in trainData:
        cnt += 1
        result += row[2]
    return result / cnt


def InerProduct(v1, v2):
    result = 0
    for i in range(len(v1)):
        result += v1[i] * v2[i]
    return result


def PredictScore(av, bu, bi, pu, qi, btcj, contextCondition):
    pScore = av + bu + bi + InerProduct(pu, qi)
    for key, value in btcj.iteritems():
        if contextCondition[key] >= 0:
            pScore += value[int(contextCondition[key])]
    if pScore < 1:
        pScore = 1
    elif pScore > 5:
        pScore = 5
    return pScore


def MAE(testData, av, bu, bi, pu, qi, btcj, contextCondition):
    global userIdDict
    global itemIdDict
    global categoryIdDict
    cnt = 0
    mae = 0.0
    for arr in testData:
        cnt += 1
        uid = userIdDict[arr[0]]
        iid = itemIdDict[arr[1]]
        cid = categoryIdDict[arr[23]]
        contextCondition = list()
        for i in range(7, 19):
            contextCondition.append(arr[i] - 1)
        pScore = PredictScore(av, bu[uid], bi[iid], pu[uid], qi[iid], btcj[cid], contextCondition)
        tScore = int(arr[2])
        mae += abs(tScore - pScore)
    fi.close()
    return mae / cnt


def CAMF_CC(trainData, validationData, testData):
    global userIdDict
    global itemIdDict
    global categoryIdDict
    global userNum
    global itemNum
    global factorNum
    global learnRate
    global regularization
    global categoryNum

    averageScore = Average(trainData)
    bu = [0.0 for i in range(userNum)]
    bi = [0.0 for i in range(itemNum)]
    temp = math.sqrt(factorNum)
    qi = [[(0.1 * random.random() / temp) for j in range(factorNum)] for i in range(itemNum)]
    pu = [[(0.1 * random.random() / temp) for j in range(factorNum)] for i in range(userNum)]
    btcj = list()
    for i in range(categoryNum):
        contextDict = dict()
        for j in range(len(contextNum)):
            contextDict[j] = [0.0 for z in range(contextNum[j])]
        btcj.append(contextDict)

    preMAE = 10000.0
    for step in range(100):
        for arr in trainData:
            uid = userIdDict[arr[0]]
            iid = itemIdDict[arr[1]]
            cid = categoryIdDict[arr[23]]
            score = int(arr[2])
            contextCondition = list()
            for i in range(7, 19):
                contextCondition.append(int(arr[i]) - 1)
        prediction = PredictScore(averageScore, bu[uid], bi[iid], pu[uid], qi[iid], btcj[cid], contextCondition)
        eui = score - prediction

        # update parameters
        bu[uid] += learnRate * (eui - regularization * bu[uid])
        bi[iid] += learnRate * (eui - regularization * bi[iid])

        for key, values in btcj[cid].iteritems():
            value = values[contextCondition[key]]
            value += learnRate * (eui - regularization * value)
            btcj[cid][key][contextCondition[key]] = value

        for k in range(factorNum):
            temp = pu[uid][k]    # attention here, must save the value of pu before updating
            pu[uid][k] += learnRate * (eui * qi[iid][k] - regularization * pu[uid][k])
            qi[iid][k] += learnRate * (eui * temp - regularization * qi[iid][k])

    curMAE = MAE(validationData, averageScore, bu, bi, pu, qi, btcj, contextCondition)
    print("test_MAE in step %d: %f" % (step, curMAE))
    if curMAE >= preMAE:
        break
    else:
        preMAE = curMAE

print("model generation over")
mae = MAE(testData, averageScore, bu, bi, pu, qi, btcj, contextCondition)
return mae

